In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import os
import functions

### 0) Configuração de pré-pesquisa
Aqui será configurado as informaçoes fornecidas para a pesquisa, como a data de inicio e fim da busca, tamanho da visualização e UF(Unidade Federativa) que será pesquisado os discursos.

In [2]:
#formato de data = {dd, mm, aaaa}
data_inicio = [28, 3, 2022]
data_fim = [28, 4, 2022]
#tamanho = quantidade de linhas de informações por página (limite 1788 pq? não sabemos)
tamanho = 100
current_page = 1
#Estado onde será realizada a pesquisa ('' para todos)
UF = ''

url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
      

In [3]:
html = requests.get(url)

content = html.content

soup = BeautifulSoup(content, 'html.parser')

In [4]:
quantidade_de_discursos= []
qpt = 0 #quantidade de paginas totais

for links in soup.find_all('span', attrs={'class':"visualStrong"}):
    quantidade_de_discursos.append(links.contents)
qpt= int(((int((quantidade_de_discursos[len(quantidade_de_discursos)-1][0]).replace('.','')))/tamanho)+1)
qpt
#quantidade_de_discursos

22

In [5]:

def UrlToSoup(url):
    html = requests.get(url)
    content = html.content
    return BeautifulSoup(content, 'html.parser')

### 1) Criar um data frame com as informações do site

In [6]:
url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF={UF}&dtInicio={data_inicio[0]}%2F{data_inicio[1]}%2F{data_inicio[2]}&dtFim={data_fim[0]}%2F{data_fim[1]}%2F{data_fim[2]}&txSessao=&listaTipoFala=&listaFaseSessao=&listaTipoInterv=&txAparteante=&txTexto=&txSumario=&txIndexacao=&BasePesq=plenario&CampoOrdenacao=dtSessao&PageSize={tamanho}&TipoOrdenacao=ASC&btnPesqAvan=Pesquisar'

html = requests.get(url, verify = False)

content = html.content

soup = BeautifulSoup(content, 'html.parser')

print(url)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.camara.leg.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF=&dtInicio=28%2F3%2F2022&dtFim=28%2F4%2F2022&txSessao=&listaTipoFala=&listaFaseSessao=&listaTipoInterv=&txAparteante=&txTexto=&txSumario=&txIndexacao=&BasePesq=plenario&CampoOrdenacao=dtSessao&PageSize=100&TipoOrdenacao=ASC&btnPesqAvan=Pesquisar


In [7]:
# codigo de verificação de ssl 
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [8]:
data_frame = [None]*qpt
for i in range(qpt):
    current_page = i+1
    url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
    data_frame[i] = pd.read_html(url)


In [9]:
data_frame[0][0]

,Data,Sessão,Fase,Discurso,Sumário,Orador,Hora,Publicação
0,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 15
1,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 41
2,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 44
3,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 49
4,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 53
...,...,...,...,...,...,...,...,...
95,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"REINHOLD STEPHANES JUNIOR, PSD-PR",15h52,DCD30/03/2022 PAG. 58
96,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"FRANCO CARTAFINA, PP-MG",15h56,DCD30/03/2022 PAG. 59
97,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"MARCEL VAN HATTEM, NOVO-RS",16h00,DCD30/03/2022 PAG. 60
98,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"DANIEL SILVEIRA, UNIÃO-RJ",16h04,DCD30/03/2022 PAG. 61


### 2) Criar uma lista com todos os links para os discursos

In [10]:
link_infos= []
soup = []
temp = []
for i in range(qpt):
        current_page = i+1
        url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
        soup.append(UrlToSoup(url))
        for links in soup[i].find_all('a', attrs={'title':"Íntegra do Discurso"}):
            temp.append((links['href']))
        link_infos.append(temp)
        temp = []


In [11]:
link_infos

[['TextoHTML.asp?etapa=5\r\n\t\t\t\t\t\t\t&nuSessao=013.4.56.N\r\n\t\t\t\t\t\t\t&nuQuarto=0\r\n\t\t\t\t\t\t\t&nuOrador=0\r\n\t\t\t\t\t\t\t&nuInsercao=3\r\n\t\t\t\t\t\t\t&dtHorarioQuarto=12:37\r\n\t\t\t\t\t\t\t&sgFaseSessao=OD\r\n\t\t\t\t\t\t\t&Data=28/04/2022\r\n\t\t\t\t\t\t\t&txApelido=AFONSO FLORENCE, PT-BA\r\n\t\t\t\t\t\t\t&txFaseSessao=Ordem do Dia\r\n\t\t\t\t\t\t\t&txTipoSessao=Extraordinária - CN\r\n\t\t\t\t\t\t\t&dtHoraQuarto=12:37\r\n\t\t\t\t\t\t\t&txEtapa=',
  'TextoHTML.asp?etapa=5\r\n\t\t\t\t\t\t\t&nuSessao=013.4.56.N\r\n\t\t\t\t\t\t\t&nuQuarto=0\r\n\t\t\t\t\t\t\t&nuOrador=0\r\n\t\t\t\t\t\t\t&nuInsercao=23\r\n\t\t\t\t\t\t\t&dtHorarioQuarto=12:37\r\n\t\t\t\t\t\t\t&sgFaseSessao=OD\r\n\t\t\t\t\t\t\t&Data=28/04/2022\r\n\t\t\t\t\t\t\t&txApelido=AFONSO FLORENCE, PT-BA\r\n\t\t\t\t\t\t\t&txFaseSessao=Ordem do Dia\r\n\t\t\t\t\t\t\t&txTipoSessao=Extraordinária - CN\r\n\t\t\t\t\t\t\t&dtHoraQuarto=12:37\r\n\t\t\t\t\t\t\t&txEtapa=',
  'TextoHTML.asp?etapa=5\r\n\t\t\t\t\t\t\t&nuSessao=013

In [12]:
len(link_infos[0])

100

In [13]:
linkDiscurso = []
temp = []
for i in range(qpt):
    for item in link_infos[i]:
        text = item.replace('\t','').replace('\n','').replace('\r','').replace(' ','')
        temp.append(("https://www.camara.leg.br/internet/SitaqWeb/"+text))
    linkDiscurso.append(temp)
    temp = []

In [14]:
len(linkDiscurso[0])

100

In [15]:
len(linkDiscurso[0])

100

In [16]:
linkDiscurso

[['https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuInsercao=3&dtHorarioQuarto=12:37&sgFaseSessao=OD&Data=28/04/2022&txApelido=AFONSOFLORENCE,PT-BA&txFaseSessao=OrdemdoDia&txTipoSessao=Extraordinária-CN&dtHoraQuarto=12:37&txEtapa=',
  'https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuInsercao=23&dtHorarioQuarto=12:37&sgFaseSessao=OD&Data=28/04/2022&txApelido=AFONSOFLORENCE,PT-BA&txFaseSessao=OrdemdoDia&txTipoSessao=Extraordinária-CN&dtHoraQuarto=12:37&txEtapa=',
  'https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuInsercao=27&dtHorarioQuarto=12:37&sgFaseSessao=OD&Data=28/04/2022&txApelido=AFONSOFLORENCE,PT-BA&txFaseSessao=OrdemdoDia&txTipoSessao=Extraordinária-CN&dtHoraQuarto=12:37&txEtapa=',
  'https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuIns

In [31]:
txt = []
for script in linkDiscurso[0]:
    url_script = UrlToSoup(script).find_all("font")
    txt.append((str((url_script[0].contents)) + "+" + str((url_script[1].contents))).replace('<br/>','').replace('<b>', '').replace('</b>', '').replace('\'', ''))

In [18]:
# for i in data_frame[0][0].index:
#     if not(os.path.exists(f"./discursos/{i}.txt")):
#         textfile = open(f"./discursos/{i}.txt", "x")
#         textfile.close()

In [32]:
id_list = []
id = []
for i in range(len(linkDiscurso[0])):
    id.append('nI')
    id.append((re.findall(r'(?<=nuInsercao=)\d+', linkDiscurso[0][i]))[0])
    id.append("nQ")
    id.append((re.findall(r'(?<=nuQuarto=)\d+', linkDiscurso[0][i]))[0])
    id.append("nS")
    id.append((re.findall(r'(?<=nuSessao=)\d+', linkDiscurso[0][i]))[0])
    id.append("nO")
    id.append((re.findall(r'(?<=nuOrador=)\d+', linkDiscurso[0][i]))[0])
    id_list.append("".join(id))
    id = []

In [33]:
id_list

['nI3nQ0nS013nO0',
 'nI23nQ0nS013nO0',
 'nI27nQ0nS013nO0',
 'nI33nQ0nS013nO0',
 'nI40nQ0nS013nO0',
 'nI41nQ0nS013nO0',
 'nI37nQ0nS013nO0',
 'nI46nQ0nS013nO0',
 'nI48nQ0nS013nO0',
 'nI52nQ0nS013nO0',
 'nI6nQ0nS013nO0',
 'nI1nQ0nS013nO0',
 'nI2nQ0nS013nO0',
 'nI5nQ0nS013nO0',
 'nI14nQ0nS013nO0',
 'nI22nQ0nS013nO0',
 'nI10nQ0nS013nO0',
 'nI18nQ0nS013nO0',
 'nI21nQ0nS013nO0',
 'nI35nQ0nS013nO0',
 'nI44nQ0nS013nO0',
 'nI42nQ0nS013nO0',
 'nI51nQ0nS013nO0',
 'nI60nQ0nS013nO0',
 'nI17nQ0nS013nO0',
 'nI16nQ0nS013nO0',
 'nI31nQ0nS013nO0',
 'nI56nQ0nS013nO0',
 'nI8nQ0nS013nO0',
 'nI19nQ0nS013nO0',
 'nI12nQ0nS013nO0',
 'nI25nQ0nS013nO0',
 'nI4nQ0nS013nO0',
 'nI28nQ0nS013nO0',
 'nI29nQ0nS013nO0',
 'nI7nQ0nS013nO0',
 'nI11nQ0nS013nO0',
 'nI15nQ0nS013nO0',
 'nI34nQ0nS013nO0',
 'nI45nQ0nS013nO0',
 'nI43nQ0nS013nO0',
 'nI39nQ0nS013nO0',
 'nI50nQ0nS013nO0',
 'nI47nQ0nS013nO0',
 'nI26nQ0nS013nO0',
 'nI13nQ0nS013nO0',
 'nI57nQ0nS013nO0',
 'nI58nQ0nS013nO0',
 'nI59nQ0nS013nO0',
 'nI55nQ0nS013nO0',
 'nI62nQ

In [21]:
data_frame[0][0]['Id'] = id_list

In [22]:
data_frame[0][0] = data_frame[0][0].set_index("Id")


In [34]:
for i in range(len(txt)):

    textfile = open(f"./discursos/{data_frame[0][0].index[i]}.txt", "w")
    textfile.write(txt[i] + "\n")
    textfile.close()

In [24]:
for col in data_frame[0][0]:
    print(data_frame[0][0][col].unique())

['28/04/2022' '26/04/2022' '05/04/2022' '29/03/2022']
['013.4.56.N' '011.4.56.N' '009.4.56.N' '25.2022']
['ORDEM DO DIA' 'HOMENAGEM' 'BREVES COMUNICAÇÕES']
[nan]
[nan]
['AFONSO FLORENCE, PT-BA' 'AGUINALDO RIBEIRO, PP-PB'
 'ANTONIO BRITO, PSD-BA' 'ARLINDO CHINAGLIA, PT-SP'
 'BIRA DO PINDARÉ, PSB-MA' 'CARMEN ZANOTTO, CIDADANIA-SC'
 'CHRISTIANE DE SOUZA YARED, PL-PR' 'CLAUDIO CAJADO, PP-BA'
 'DR. ZACARIAS CALIL, DEM-GO' 'FÁBIO TRAD, PSD-MS' 'HEITOR SCHUCH, PSB-RS'
 'IVAN VALENTE, PSOL-SP' 'LEÔNIDAS CRISTINO, PDT-CE'
 'MÁRCIO MACÊDO, PT-SE' 'NELHO BEZERRA, UNIÃO-CE'
 'OTTACI NASCIMENTO, SOLIDARIEDADE-RR' 'PAULO TEIXEIRA, PT-SP'
 'RICARDO BARROS, PP-PR' 'SORAYA SANTOS, PL-RJ' 'TIAGO MITRAUD, NOVO-MG'
 'VALTENIR PEREIRA, MDB-MT' 'VINICIUS CARVALHO, REPUBLICANOS-SP'
 'VIVI REIS, PSOL-PA' 'WELITON PRADO, PROS-MG'
 'GENERAL PETERNELLI, UNIÃO-SP' 'MARGARETE COELHO, PP-PI'
 'FREI ANASTACIO RIBEIRO, PT-PB' 'VALMIR ASSUNÇÃO, PT-BA'
 'ROSANGELA GOMES, REPUBLICANOS-RJ' 'GENERAL GIRÃO, PL-RN'
 'HELDER

In [25]:
#data_frame[0][0]['Id'].nunique()

## Modificando datas

In [26]:
data_frame[0][0]

,Data,Sessão,Fase,Discurso,Sumário,Orador,Hora,Publicação
Id,,,,,,,,
nI3nQ0nS013nO0,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 15
nI23nQ0nS013nO0,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 41
nI27nQ0nS013nO0,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 44
nI33nQ0nS013nO0,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 49
nI40nQ0nS013nO0,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 53
...,...,...,...,...,...,...,...,...
nI3nQ553460nS25nO3,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"REINHOLD STEPHANES JUNIOR, PSD-PR",15h52,DCD30/03/2022 PAG. 58
nI3nQ553461nS25nO3,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"FRANCO CARTAFINA, PP-MG",15h56,DCD30/03/2022 PAG. 59
nI3nQ553462nS25nO3,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"MARCEL VAN HATTEM, NOVO-RS",16h00,DCD30/03/2022 PAG. 60


In [27]:
data_frame[0][0]["Data"] = pd.to_datetime(data_frame[0][0]["Data"])
data_frame[0][0]

/home/jonasfelipe/.local/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '28/04/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/home/jonasfelipe/.local/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '26/04/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/home/jonasfelipe/.local/lib/python3.8/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '29/03/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)


,Data,Sessão,Fase,Discurso,Sumário,Orador,Hora,Publicação
Id,,,,,,,,
nI3nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 15
nI23nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 41
nI27nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 44
nI33nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 49
nI40nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 05/05/2022 PAG. 53
...,...,...,...,...,...,...,...,...
nI3nQ553460nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"REINHOLD STEPHANES JUNIOR, PSD-PR",15h52,DCD30/03/2022 PAG. 58
nI3nQ553461nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"FRANCO CARTAFINA, PP-MG",15h56,DCD30/03/2022 PAG. 59
nI3nQ553462nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"MARCEL VAN HATTEM, NOVO-RS",16h00,DCD30/03/2022 PAG. 60


In [28]:
data_frame_orador = [x.split(", ") for x in data_frame[0][0]["Orador"]]
data_frame_orador_UF = [i[1].split("-") for i in data_frame_orador]
data_frame_orador_final = [{"Orador": data_frame_orador[x][0], "Partido": data_frame_orador_UF[x][0], "UF": data_frame_orador_UF[x][1]} for x in range(len(data_frame_orador))]

orador = []
partido = []
uf = []
for i in data_frame_orador_final:
    orador.append(i['Orador'])
    partido.append(i['Partido'])
    uf.append(i['UF'])

data_frame[0][0]["Orador"] = orador
data_frame[0][0]["Partido"] = partido
data_frame[0][0]["UF"] = uf

data_frame[0][0]

,Data,Sessão,Fase,Discurso,Sumário,Orador,Hora,Publicação,Partido,UF
Id,,,,,,,,,,
nI3nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,12h37,DCN 05/05/2022 PAG. 15,PT,BA
nI23nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,12h37,DCN 05/05/2022 PAG. 41,PT,BA
nI27nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,12h37,DCN 05/05/2022 PAG. 44,PT,BA
nI33nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,12h37,DCN 05/05/2022 PAG. 49,PT,BA
nI40nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,12h37,DCN 05/05/2022 PAG. 53,PT,BA
...,...,...,...,...,...,...,...,...,...,...
nI3nQ553460nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,REINHOLD STEPHANES JUNIOR,15h52,DCD30/03/2022 PAG. 58,PSD,PR
nI3nQ553461nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,FRANCO CARTAFINA,15h56,DCD30/03/2022 PAG. 59,PP,MG
nI3nQ553462nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,MARCEL VAN HATTEM,16h00,DCD30/03/2022 PAG. 60,NOVO,RS


In [29]:
['Data', 'Sessão', 'Fase', 'Discurso', 'Sumário', 'Orador', 'Hora', 'Publicação', 'Partido', 'UF']

data_frame[0][0] = data_frame[0][0][['Data', 'Sessão', 'Fase', 'Discurso', 'Sumário', 'Orador', 'Partido', 'UF', 'Hora', 'Publicação']]
data_frame[0][0]

,Data,Sessão,Fase,Discurso,Sumário,Orador,Partido,UF,Hora,Publicação
Id,,,,,,,,,,
nI3nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 15
nI23nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 41
nI27nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 44
nI33nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 49
nI40nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 53
...,...,...,...,...,...,...,...,...,...,...
nI3nQ553460nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,REINHOLD STEPHANES JUNIOR,PSD,PR,15h52,DCD30/03/2022 PAG. 58
nI3nQ553461nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,FRANCO CARTAFINA,PP,MG,15h56,DCD30/03/2022 PAG. 59
nI3nQ553462nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,MARCEL VAN HATTEM,NOVO,RS,16h00,DCD30/03/2022 PAG. 60


In [30]:
data_frame[0][0].drop("Sumário", axis = 1)

,Data,Sessão,Fase,Discurso,Orador,Partido,UF,Hora,Publicação
Id,,,,,,,,,
nI3nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 15
nI23nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 41
nI27nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 44
nI33nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 49
nI40nQ0nS013nO0,2022-04-28,013.4.56.N,ORDEM DO DIA,NaN,AFONSO FLORENCE,PT,BA,12h37,DCN 05/05/2022 PAG. 53
...,...,...,...,...,...,...,...,...,...
nI3nQ553460nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,REINHOLD STEPHANES JUNIOR,PSD,PR,15h52,DCD30/03/2022 PAG. 58
nI3nQ553461nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,FRANCO CARTAFINA,PP,MG,15h56,DCD30/03/2022 PAG. 59
nI3nQ553462nS25nO3,2022-03-29,25.2022,BREVES COMUNICAÇÕES,NaN,MARCEL VAN HATTEM,NOVO,RS,16h00,DCD30/03/2022 PAG. 60
